In [30]:
import numpy as np

In [31]:
np.set_printoptions(suppress=True, linewidth=300,precision=2)

In [32]:
data0=np.genfromtxt("loan-data.csv",delimiter=";", skip_header=1, autostrip=True)
print(data0)

[[48010226.           nan    35000.   ...         nan         nan     9452.96]
 [57693261.           nan    30000.   ...         nan         nan     4679.7 ]
 [59432726.           nan    15000.   ...         nan         nan     1969.83]
 ...
 [50415990.           nan    10000.   ...         nan         nan     2185.64]
 [46154151.           nan         nan ...         nan         nan     3199.4 ]
 [66055249.           nan    10000.   ...         nan         nan      301.9 ]]


Separ tabelul in 2:unul cu stringuri (string_cols) si unul cu numere (numeric_cols)

In [33]:
numeric_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0))==False)[0]
string_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0)))[0]
print(numeric_cols_indices)
print(string_cols_indices)

[ 0  2  4  6  7 13]
[ 1  3  5  8  9 10 11 12]


C:\Users\EMELNIAP8\AppData\Local\Temp\ipykernel_1580\4260221152.py:1: RuntimeWarning: Mean of empty slice
  numeric_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0))==False)[0]
C:\Users\EMELNIAP8\AppData\Local\Temp\ipykernel_1580\4260221152.py:2: RuntimeWarning: Mean of empty slice
  string_cols_indices=np.where(np.isnan(np.nanmean(data0, axis=0)))[0]


In [34]:
numeric_cols=np.genfromtxt("loan-data.csv",delimiter=";", skip_header=1, usecols=numeric_cols_indices,autostrip=True)
string_cols=np.genfromtxt("loan-data.csv",delimiter=";", skip_header=1, usecols=string_cols_indices, dtype=np.str_,autostrip=True)
print(numeric_cols, end="\n\n")
print(string_cols)

[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.           nan      938.57     4679.7 ]
 [59432726.      15000.      15000.           nan      494.86     1969.83]
 ...
 [50415990.      10000.      10000.           nan         nan     2185.64]
 [46154151.           nan    10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.           nan      309.97      301.9 ]]

[['May-15' 'Current' '36 months' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA']
 ['' 'Current' '36 months' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261' 'NY']
 ['Sep-15' 'Current' '36 months' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726' 'PA']
 ...
 ['Jun-15' 'Current' '36 months' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990' 'CA']
 ['Apr-15' 'Current' '36 m

In [35]:
numeric_header=np.genfromtxt("loan-data.csv",delimiter=";", skip_footer=numeric_cols.shape[0], usecols=numeric_cols_indices,autostrip=True, dtype=np.str_)
string_header=np.genfromtxt("loan-data.csv",delimiter=";", skip_footer=string_cols.shape[0], usecols=string_cols_indices,autostrip=True, dtype=np.str_)
print(numeric_header)
print(string_header)

['id' 'loan_amount' 'funded_amount' 'int_rate' 'installment' 'total_payment']
['issue_date' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']


"Issue Date"

In [36]:
print(np.unique(string_cols[:,0]))

['' 'Apr-15' 'Aug-15' 'Dec-15' 'Feb-15' 'Jan-15' 'Jul-15' 'Jun-15' 'Mar-15' 'May-15' 'Nov-15' 'Oct-15' 'Sep-15']


Elimin '-15'din coloana 'issue-date' de pe toate liniile, pentru ca are aceeasi valoare pe toate liniile

In [37]:
string_cols[:,0] = np.char.replace(string_cols[:,0],  '-15',  '')
print(np.unique(string_cols[:,0]))

['' 'Apr' 'Aug' 'Dec' 'Feb' 'Jan' 'Jul' 'Jun' 'Mar' 'May' 'Nov' 'Oct' 'Sep']


Inlocuiesc fiecare luna cu echivalentul ei numeric ('Jan'->'1')

In [38]:
months=np.array(["", "Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct", "Nov", "Dec"])

In [39]:
for i in range(months.shape[0]):
    string_cols[:,0]=np.where(string_cols[:,0]==months[i], str(i), string_cols[:,0])
    
print(np.unique(string_cols[:,0]))

['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']


In [40]:
string_header[0]="issue_month"

"Loan Status"

In [41]:
print(np.unique(string_cols[:,1]))
print(string_cols[:,1])

['' 'Charged Off' 'Current' 'Default' 'Fully Paid' 'In Grace Period' 'Issued' 'Late (16-30 days)' 'Late (31-120 days)']
['Current' 'Current' 'Current' ... 'Current' 'Current' 'Current']


"Charged Off" = debitorul nu a platit o perioada lunga si banca considera creditul pierdere, inchide oficial creditul in contabilitate "Current" = imprumutul este la zi, toate platile au fost facute conform graficului "Default" = debitorul este in incapacitate de plata "Fully Paid" = creditul a fost platit integral "In Grace Period" = debitorul a intarziat plata, dar se afla inca in perioada de gratie "Issued" = creditul a fost acordat recent, banii au fost transferati catre debitor "Late" = plata intarziata

Pentru a determina daca un candidat e stabil financiar si eligibil pentru credit, putem foosi doar 2 valori:0 si 1. ["charged off", "default", "late (31-120 days)", no data]->0, iar ["current", "fully paid", "in grace period", "issued", "late(16-30 days)"]->1


In [42]:
good_status=np.array(["Current", "Fully Paid", "In Grace Period", "Issued", "Late (16-30 days)"])
string_cols[:,1]=np.where(np.isin(string_cols[:,1], good_status), "1" , "0")

In [43]:
print(np.unique(string_cols[:,1]))

['0' '1']


"Term"

In [44]:
print(np.unique(string_cols[:,2]))

['' '36 months' '60 months']


In [45]:
string_cols[:,2] = np.char.replace(string_cols[:,2],  ' months',  '')
print(np.unique(string_cols[:,2]))

['' '36' '60']


Redenumesc coloana, ca se fie clar ca acestea sun luni:

In [46]:
string_header[2]="term_months"

In loc de datele lipsa, setez 60 luni (worst case):

In [47]:
string_cols[:,2] = np.where(string_cols[:,2]=="", "60", string_cols[:,2])
print(np.unique(string_cols[:,2]))

['36' '60']


"Grade" , "Subgrade"= clasa de risc a unui debitor

In [48]:
print(np.unique(string_cols[:,3]))
print(np.unique(string_cols[:,4]))

['' 'A' 'B' 'C' 'D' 'E' 'F' 'G']
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [49]:
string_cols[:,4]=np.where((string_cols[:,4]=="") & (string_cols[:,3]!=""), np.char.add(string_cols[:,3],"5"),string_cols[:,4])
print(np.unique(string_cols[:,4]))

['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [50]:
string_cols[:,4]=np.where(string_cols[:,4]=="", "G5",string_cols[:,4])
print(np.unique(string_cols[:,4]))

['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [51]:
string_cols=np.delete(string_cols,3,axis=1)
string_header=np.delete(string_header,3)
print(string_cols, end="\n\n")
print(string_header)

[['5' '1' '36' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA']
 ['0' '1' '36' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261' 'NY']
 ['9' '1' '36' ... 'Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726' 'PA']
 ...
 ['6' '1' '36' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990' 'CA']
 ['4' '1' '36' ... 'Source Verified' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151' 'OH']
 ['12' '1' '36' ... '' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249' 'IL']]

['issue_month' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'url' 'addr_state']


Asociez cate un numar pentru fiecare subgrade:

In [52]:
subgrade_keys=np.unique(string_cols[:,3])
subgrade_values=range(1,np.unique(string_cols[:,3]).shape[0]+1)
subgrade_dict=dict(zip(subgrade_keys,subgrade_values))
print(subgrade_dict)

{np.str_('A1'): 1, np.str_('A2'): 2, np.str_('A3'): 3, np.str_('A4'): 4, np.str_('A5'): 5, np.str_('B1'): 6, np.str_('B2'): 7, np.str_('B3'): 8, np.str_('B4'): 9, np.str_('B5'): 10, np.str_('C1'): 11, np.str_('C2'): 12, np.str_('C3'): 13, np.str_('C4'): 14, np.str_('C5'): 15, np.str_('D1'): 16, np.str_('D2'): 17, np.str_('D3'): 18, np.str_('D4'): 19, np.str_('D5'): 20, np.str_('E1'): 21, np.str_('E2'): 22, np.str_('E3'): 23, np.str_('E4'): 24, np.str_('E5'): 25, np.str_('F1'): 26, np.str_('F2'): 27, np.str_('F3'): 28, np.str_('F4'): 29, np.str_('F5'): 30, np.str_('G1'): 31, np.str_('G2'): 32, np.str_('G3'): 33, np.str_('G4'): 34, np.str_('G5'): 35}


In [53]:
string_cols[:,3]=[str(subgrade_dict[key]) for key in string_cols[:,3]]

In [54]:
print(np.unique(string_cols[:,3]))
print(string_cols[:,3])

['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '34' '35' '4' '5' '6' '7' '8' '9']
['13' '5' '10' ... '5' '17' '4']


"Verification Status"

In [55]:
print(np.unique(string_cols[:,4]))

['' 'Not Verified' 'Source Verified' 'Verified']


In [56]:
string_cols[:,4] = np.where((string_cols[:,4] == '') | (string_cols[:,4] == 'Not Verified'), 0, 1)
print(np.unique(string_cols[:,4]))

['0' '1']


"URL"

In [57]:
print(np.unique(string_cols[:,5]))

['https://www.lendingclub.com/browse/loanDetail.action?loan_id=12606806' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=13026045' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1312426' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8138291'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8214572' 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=849994']


In [58]:
string_cols[:,5]=np.chararray.strip(string_cols[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")
print(string_cols[:,5])

['48010226' '57693261' '59432726' ... '50415990' '46154151' '66055249']


C:\Users\EMELNIAP8\AppData\Local\Temp\ipykernel_1580\3706755230.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  string_cols[:,5]=np.chararray.strip(string_cols[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")


In [59]:
numeric_cols[:,0].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], shape=(10000,), dtype=int32)

In [60]:
string_cols[:,5].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], shape=(10000,), dtype=int32)

In [61]:
np.array_equal(numeric_cols[:,0].astype(dtype = np.int32), string_cols[:,5].astype(dtype = np.int32))

True

In [64]:
string_cols = np.delete(string_cols, 5, axis = 1)
string_header = np.delete(string_header, 5)

In [65]:
print(string_cols[:,5])
print(string_header)
print(numeric_cols[:,0])
print(numeric_header)

IndexError: index 5 is out of bounds for axis 1 with size 5